<a href="https://colab.research.google.com/github/Yeen000/Aiffel/blob/master/Main_Quest_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MAIN QUEST 3
* 기본 설정에서, 몬스터의 설정을 조금 바꾸고 무기를 설정할 수 있게 하였음.
* 선택한 무기에 따라 가할 수 있는 Damage가 달라짐.
* 몬스터의 수는 너무 많으면 게임이 루즈해질 수 있을 것 같아 3으로 고정

In [22]:
import random

# Character 클래스 정의
class Character:
    def __init__(self, name, level, hp, attack, defense):
        self.name = name
        self.level = level
        self.hp = hp
        self.attack = attack
        self.defense = defense

    def is_alive(self):
        return self.hp > 0

    def take_damage(self, damage):
        actual_damage = max(damage - self.defense, 0)  # 방어력 적용 후 최소 0 이상 피해
        if actual_damage == 0:
            print(f"{self.name}은(는) 공격을 완전히 막아냈습니다!")
        else:
            self.hp -= actual_damage

    def attack_target(self, target):
        try:
            choice_weapon = int(input(">> 어떤 무기를 사용하시겠습니까? 번호를 입력해 주세요 (1:axe, 2:sword, 3:gun): "))
        except ValueError:
            print("잘못된 입력입니다. 기본 공격으로 시도하겠습니다.")
            choice_weapon = 2  # 기본값을 sword로 설정

        weapon_multiplier = {1: 0.7, 2: 1.0, 3: 2.5}  # 무기 별 공격 배율
        damage = random.randint(self.attack // 2, self.attack) * weapon_multiplier.get(choice_weapon, 1.0)

        target.take_damage(int(damage))
        return int(damage)  # 가한 피해 반환

    def monster_attack_target(self, target):
        damage = random.randint(1, self.attack)
        target.take_damage(damage)
        return damage  # 가한 데미지를 반환



# Player 클래스 정의
class Player(Character):
    def __init__(self, name):
        super().__init__(name, level=1, hp=100, attack=25, defense=5)
        self.exp = 0

    def gain_experience(self, exp):
        self.exp += exp
        if self.exp >= 50:
            self.level_up()
            print(f"{self.name}의 레벨이 {self.level}이 되었습니다!")

    def level_up(self):
        self.level += 1
        self.attack += 10
        self.defense += 5
        self.hp = 100  # 레벨업하면 체력 회복
        self.exp -= 50  # 경험치 차감


# Monster 클래스 정의
class Monster(Character):
    def __init__(self, name, level):
        hp = random.randint(20, 30) + (level * 10)  # 레벨마다 추가 체력
        attack = random.randint(10, 20) + (level * 10)  # 레벨마다 추가 공격력
        defense = random.randint(2, 5) + (level * 5)  # 방어력 증가
        super().__init__(name, level, hp, attack, defense)


# Battle 함수 정의
def battle(player, monster):
    print(f"\n🦸‍♂️영웅 \n이름: {player.name}\n레벨: {player.level}\n체력: {player.hp} \n공격력: {player.attack} \n방어력: {player.defense} ")
    print(f"\n👹몬스터 \n이름: {monster.name}\n레벨: {monster.level}\n체력: {monster.hp} \n공격력: {monster.attack} \n방어력: {monster.defense} ")
    print(f"\n⚔️ {monster.name} (레벨 {monster.level})과의 전투를 시작합니다!")

    while player.is_alive() and monster.is_alive():
        # 플레이어의 공격
        print("\n🦸‍♂️ 플레이어의 턴!")
        player_damage = player.attack_target(monster)
        print(f"{player.name}이(가) {monster.name}에게 {player_damage}만큼의 공격을 시도하여 {player_damage - monster.defense}만큼의 피해를 입히는데 성공했습니다! ")
        print(f"몬스터의 남은 체력: {monster.hp}")

        if not monster.is_alive():
            print("-"*23)
            print(f"🎉 {monster.name}을(를) 처치했습니다!")
            exp_gained = monster.level * 20
            player.gain_experience(exp_gained)
            print(f"{player.name}는 {exp_gained}의 경험치를 얻었습니다.")
            print("-"*23)
            return True  # 승리 시 True 반환

        # 몬스터의 공격
        print("\n👹 몬스터의 턴!")
        monster_damage = monster.monster_attack_target(player)
        print(f"{monster.name}이(가) {player.name}에게 {monster_damage}만큼의 공격을 시도하여 {monster_damage - player.defense}만큼의 피해를 입히는데 성공했습니다!")
        print(f"영웅의 남은 체력: {player.hp}")

        if not player.is_alive():
            print("\n💀 전투 패배..")
            return False  # 패배 시 False 반환

    return True  # 기본적으로 승리 시 True 반환

def main():
    monster_dict = {"슬라임": 1, "고블린": 2, "오크": 3}
    player_name = input("⚔️영웅의 이름을 입력하세요⚔️: ")
    player = Player(player_name)

    # 모든 몬스터를 처치했는지 확인하는 변수
    all_monsters_defeated = True

    for monster_name, level in monster_dict.items():
        monster = Monster(monster_name, level)

        battle_result = battle(player, monster)
        if not battle_result:  # 플레이어 사망
            print("\n💀 플레이어가 사망했습니다. 게임을 종료합니다.")
            all_monsters_defeated = False  # 모든 몬스터 처치 실패
            break  # 루프 종료

    # 모든 몬스터를 물리쳤을 때만 게임 클리어 출력
    if all_monsters_defeated:
        print("\n🎉 모든 몬스터를 물리쳤습니다! 게임 클리어! 🎉")


if __name__ == "__main__":
    main()

# # 테스트 실행
# player = Player("영웅")
# monster = Monster("고블린", level=3)

# battle(player, monster)

⚔️영웅의 이름을 입력하세요⚔️: 옌

🦸‍♂️영웅 
이름: 옌
레벨: 1
체력: 100 
공격력: 25 
방어력: 5 

👹몬스터 
이름: 슬라임
레벨: 1
체력: 34 
공격력: 18 
방어력: 7 

⚔️ 슬라임 (레벨 1)과의 전투를 시작합니다!

🦸‍♂️ 플레이어의 턴!
>> 어떤 무기를 사용하시겠습니까? 번호를 입력해 주세요 (1:axe, 2:sword, 3:gun): 2
옌이(가) 슬라임에게 22만큼의 공격을 시도하여 15만큼의 피해를 입히는데 성공했습니다! 
몬스터의 남은 체력: 19

👹 몬스터의 턴!
슬라임이(가) 옌에게 17만큼의 공격을 시도하여 12만큼의 피해를 입히는데 성공했습니다!
영웅의 남은 체력: 88

🦸‍♂️ 플레이어의 턴!
>> 어떤 무기를 사용하시겠습니까? 번호를 입력해 주세요 (1:axe, 2:sword, 3:gun): 1
옌이(가) 슬라임에게 15만큼의 공격을 시도하여 8만큼의 피해를 입히는데 성공했습니다! 
몬스터의 남은 체력: 11

👹 몬스터의 턴!
슬라임이(가) 옌에게 15만큼의 공격을 시도하여 10만큼의 피해를 입히는데 성공했습니다!
영웅의 남은 체력: 78

🦸‍♂️ 플레이어의 턴!
>> 어떤 무기를 사용하시겠습니까? 번호를 입력해 주세요 (1:axe, 2:sword, 3:gun): 3
옌이(가) 슬라임에게 60만큼의 공격을 시도하여 53만큼의 피해를 입히는데 성공했습니다! 
몬스터의 남은 체력: -42
-----------------------
🎉 슬라임을(를) 처치했습니다!
옌는 20의 경험치를 얻었습니다.
-----------------------

🦸‍♂️영웅 
이름: 옌
레벨: 1
체력: 78 
공격력: 25 
방어력: 5 

👹몬스터 
이름: 고블린
레벨: 2
체력: 44 
공격력: 21 
방어력: 8 

⚔️ 고블린 (레벨 2)과의 전투를 시작합니다!

🦸‍♂️ 플레이어의 턴!
>> 어떤 무기를 사용하시겠습니까? 번호를 입력해 주세요 (1:axe, 2:sword, 3:gun): 2
옌이(가) 고블린

## 프로젝트 회고
* 클래스를 상속하고, 여러 클래스와 함수를 오가며 변수를 사용하다보니 많이 헷갈렸다. 연습을 더 많이 해봐야 할 것 같음!
* 코드를 짜는 능력도 중요하지만 기획하는 능력이 중요한 것 같음. 로직을 먼저 정교하게 기획해두고 코드를 짜는게 더 멋진 프로그램을 만들 수 있는 것 같다.
